# Kuramoto Phase-Locking Simulation (Brain Substrate Analogy)

This notebook demonstrates **phase locking** in coupled oscillators using the **Kuramoto model**, and connects the results to neural synchronization and ABE-inspired external driving.

**Scenarios:**
1. Two-oscillator phase difference dynamics (analytical intuition).
2. N-oscillator Kuramoto network with heterogeneous natural frequencies.
3. Effect of coupling strength K on synchronization (order parameter r(t)).
4. Entrainment by an external drive (model of field/optic entrainment).
5. Optional: Additive phase noise (robustness to perturbations).

> No seaborn; matplotlib only, single-plot figures per the constraints.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Utilities
def kuramoto_step(theta, omega, K, dt):
    '''
    One Euler step for the Kuramoto model:
      dθ_i/dt = ω_i + (K/N) * sum_j sin(θ_j - θ_i)
    '''
    N = len(theta)
    diff = theta.reshape((N,1)) - theta.reshape((1,N))
    coupling = (K/N) * np.sum(np.sin(-diff), axis=1)  # -diff -> (θ_j - θ_i)
    return theta + dt * (omega + coupling)

def order_parameter(theta):
    '''Return magnitude r and mean phase psi of the complex order parameter'''
    z = np.exp(1j*theta)
    r = np.abs(np.mean(z))
    psi = np.angle(np.mean(z))
    return r, psi

rng = np.random.default_rng(1234)

## 1) Two Oscillators: Phase Difference Dynamics

For two oscillators with natural frequencies ω1, ω2, the phase difference Δφ = θ1 − θ2 follows:
d(Δφ)/dt = Δω − K·sin(Δφ)

Stable phase locking occurs when |Δω| ≤ K and the system converges to a fixed Δφ* where sin(Δφ*) = Δω / K.


In [ ]:
# Two-oscillator demo
dt = 0.001
T = 5.0
steps = int(T/dt)

omega1 = 2*np.pi*10.0   # 10 Hz
omega2 = 2*np.pi*9.5    # 9.5 Hz
Dw = omega1 - omega2

Ks = [0.0, abs(Dw)*0.5, abs(Dw)*1.2]  # weak, threshold-ish, strong coupling

plt.figure()
for K in Ks:
    th1 = 2*np.pi*rng.random()
    th2 = 2*np.pi*rng.random()
    dphi_traj = []
    for _ in range(steps):
        dth1 = omega1 + (K/2.0)*np.sin(th2 - th1)
        dth2 = omega2 + (K/2.0)*np.sin(th1 - th2)
        th1 += dt*dth1
        th2 += dt*dth2
        dphi = (th1 - th2 + np.pi)%(2*np.pi) - np.pi
        dphi_traj.append(dphi)
    t = np.linspace(0, T, steps)
    plt.plot(t, np.unwrap(dphi_traj), label=f'K={K:.2f} rad/s')

plt.xlabel('Time (s)')
plt.ylabel('Phase difference Δφ (rad, unwrapped)')
plt.title('Two-Oscillator Phase Difference Dynamics')
plt.legend()
plt.show()

## 2) N Oscillators: Spontaneous Synchronization

We simulate N oscillators with natural frequencies drawn from a normal distribution. As K increases, the order parameter r(t) rises toward 1, indicating synchronization.


In [ ]:
# N-oscillator Kuramoto simulation
N = 200
dt = 0.002
T = 5.0
steps = int(T/dt)

# Natural frequencies (rad/s), centered at ~40 Hz to echo gamma-band
mean_hz, std_hz = 40.0, 3.0
omega = 2*np.pi*rng.normal(mean_hz, std_hz, size=N)

# Initial phases
theta0 = 2*np.pi*rng.random(N)

def run_kuramoto(K):
    theta = theta0.copy()
    r_series = []
    for _ in range(steps):
        theta = kuramoto_step(theta, omega, K, dt)
        r, _ = order_parameter(theta)
        r_series.append(r)
    return np.array(r_series)

Ksweep = [0.0, 10.0, 50.0, 150.0]  # rad/s coupling strengths (illustrative)
plt.figure()
for K in Ksweep:
    r_series = run_kuramoto(K)
    t = np.linspace(0, T, steps)
    plt.plot(t, r_series, label=f'K={K:.0f}')
plt.xlabel('Time (s)')
plt.ylabel('Order parameter r(t)')
plt.title('Kuramoto Synchronization vs Coupling Strength')
plt.legend()
plt.show()

## 3) Entrainment by External Drive (Field/Optic Analogy)

We add a global drive term to model weak field/optic entrainment:
dθ_i/dt = ω_i + (K/N) Σ_j sin(θ_j − θ_i) + ε sin(Ω t − θ_i)
where Ω is the drive frequency and ε is the drive strength.


In [ ]:
def kuramoto_step_driven(theta, omega, K, dt, t, eps, Omega):
    N = len(theta)
    diff = theta.reshape((N,1)) - theta.reshape((1,N))
    coupling = (K/N) * np.sum(np.sin(-diff), axis=1)
    drive = eps * np.sin(Omega*t - theta)
    return theta + dt*(omega + coupling + drive)

N = 200
dt = 0.001
T = 3.0
steps = int(T/dt)

omega = 2*np.pi*rng.normal(10.0, 1.0, size=N)  # ~10 Hz alpha-like
theta = 2*np.pi*rng.random(N)

K = 30.0      # moderate coupling
eps = 5.0     # weak external drive
Omega = 2*np.pi*10.0  # 10 Hz drive

r_series = []
psi_series = []
for k in range(steps):
    t = k*dt
    theta = kuramoto_step_driven(theta, omega, K, dt, t, eps, Omega)
    r, psi = order_parameter(theta)
    r_series.append(r)
    psi_series.append(psi)

t = np.linspace(0, T, steps)
plt.figure()
plt.plot(t, r_series)
plt.xlabel('Time (s)')
plt.ylabel('Order parameter r(t)')
plt.title('Driven Kuramoto Network: Entrainment Increases r(t)')
plt.show()

## 4) Noise Robustness

We add additive phase noise η_i(t) ~ N(0, σ²) to show how synchronization withstands perturbations.


In [ ]:
def kuramoto_step_driven_noisy(theta, omega, K, dt, t, eps, Omega, sigma):
    N = len(theta)
    diff = theta.reshape((N,1)) - theta.reshape((1,N))
    coupling = (K/N) * np.sum(np.sin(-diff), axis=1)
    drive = eps * np.sin(Omega*t - theta)
    noise = np.sqrt(dt) * sigma * rng.standard_normal(N)
    return theta + dt*(omega + coupling + drive) + noise

N = 200
dt = 0.001
T = 3.0
steps = int(T/dt)

omega = 2*np.pi*rng.normal(10.0, 2.0, size=N)
theta = 2*np.pi*rng.random(N)

K = 40.0
eps = 4.0
Omega = 2*np.pi*10.0
sigma_vals = [0.0, 0.5, 1.0]  # increasing noise

plt.figure()
for sigma in sigma_vals:
    theta_local = theta.copy()
    r_series = []
    for k in range(steps):
        t = k*dt
        theta_local = kuramoto_step_driven_noisy(theta_local, omega, K, dt, t, eps, Omega, sigma)
        r, _ = order_parameter(theta_local)
        r_series.append(r)
    plt.plot(np.linspace(0, T, steps), r_series, label=f'σ={sigma}')

plt.xlabel('Time (s)')
plt.ylabel('Order parameter r(t)')
plt.title('Driven Kuramoto with Noise: r(t) vs σ')
plt.legend()
plt.show()

## Interpretation & Neural/ABE Context

- As coupling K increases, oscillators synchronize, raising r(t) toward 1.
- A weak external drive (optic/EM field) can entrain the network, increasing r(t) even with moderate K.
- Noise reduces synchronization but does not eliminate it unless large; this mirrors brain data where coherence persists despite fluctuations.
- In a brain substrate, such phase locking underpins communication-through-coherence. An external drive acts like ε sin(Ω t − θ_i), nudging phases into alignment.
- ABE-inspired views treat the drive as shaping a phase landscape/reference; even a subtle, coherent reference can aid global phase alignment.

Try experimenting with: drive frequency detuning (Ω != mean ω), time-varying K(t), or multi-cluster frequency distributions.
